In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [12]:
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[ 1.1731, -0.2886,  0.4954,  ..., -0.8249, -1.3956,  0.8999],
        [ 0.9853, -0.9168,  0.1590,  ..., -0.4504, -0.3755,  0.1219],
        [-0.1647,  0.2255, -0.7531,  ..., -0.5301,  0.5368,  0.2107],
        ...,
        [ 0.5394,  0.5597, -2.3577,  ...,  2.3562,  1.2013,  1.1631],
        [ 0.0118, -0.1316, -1.0003,  ..., -0.5087,  0.1197, -1.2568],
        [-0.3816,  0.4277, -0.0862,  ...,  1.3711, -1.5236,  0.0994]],
       requires_grad=True)

## activation으로 relu사용!

In [8]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 128.798461914
Epoch: 0002 cost = 35.974052429
Epoch: 0003 cost = 23.062042236
Epoch: 0004 cost = 16.088733673
Epoch: 0005 cost = 11.609772682
Epoch: 0006 cost = 8.515786171
Epoch: 0007 cost = 6.458452225
Epoch: 0008 cost = 4.840959549
Epoch: 0009 cost = 3.644506693
Epoch: 0010 cost = 2.710587502
Epoch: 0011 cost = 2.119611263
Epoch: 0012 cost = 1.637325048
Epoch: 0013 cost = 1.266443372
Epoch: 0014 cost = 1.011371255
Epoch: 0015 cost = 0.817778230
Learning finished


In [11]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

C:\Users\van13\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\van13\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.947700023651123
Label:  9
Prediction:  9
